In [1]:
import dpo
from datasets import load_dataset
from torch.optim import AdamW
from transformers import (
    AutoModelForCausalLM,
    
)

model_id = "meta-llama/Meta-Llama-3-8B"


model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            load_in_4bit=True
        )

ref_model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            load_in_4bit=True

        )



The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
ds = load_dataset("HumanLLMs/Human-Like-DPO-Dataset")

# 訓練データのサブセットを選択
dataset = ds['train'].shuffle(seed=42).select(range(100))
beta = 1.0

dpo_model = dpo.DPO(model_id, model,ref_model,beta)
optimizer = AdamW(dpo_model.model.parameters(), lr=1e-5)

In [ ]:

# トレーニング実行
dpo_model.train(dataset, optimizer, num_epochs=5)

You shouldn't move a model that is dispatched using accelerate hooks.
You shouldn't move a model that is dispatched using accelerate hooks.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

c:\Users\otsuka\anaconda3\envs\ptorch\Lib\site-packages\bitsandbytes\nn\modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [ ]:
ds['train'][0]

{'prompt': 'Oh, I just saw the best meme - have you seen it?',
 'chosen': "😂 Ah, no I haven't! I'm dying to know, what's the meme about? Is it a funny cat or a ridiculous situation? Spill the beans! 🤣",
 'rejected': "I'm an artificial intelligence language model, I don't have personal experiences or opinions. However, I can provide you with information on highly-rated and critically acclaimed films, as well as recommendations based on specific genres or themes. Would you like me to suggest some notable movies or discuss a particular genre of interest?"}

In [ ]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(model_id)

input_ids = tokenizer.encode(ds['train'][0]["prompt"], return_tensors="pt").to(dpo_model.model.device)


output =dpo_model.model.generate(input_ids,max_new_tokens=128)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
print(tokenizer.decode(output[0], skip_special_tokens=True))


Oh, I just saw the best meme - have you seen it? - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

input_ids = tokenizer.encode(ds['train'][0]["prompt"], return_tensors="pt").to(dpo_model.ref_model.device)


output =dpo_model.ref_model.generate(input_ids,max_new_tokens=128)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
print(tokenizer.decode(output[0], skip_special_tokens=True))


Oh, I just saw the best meme - have you seen it? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all time? - of all
